Я решаю данную задачу используя библиотеку LightAutoML [ссылка](https://github.com/sb-ai-lab/LightAutoML). 

# Условия

## Описание задачи
В вашем распоряжении есть набор изображений, которые состоят из нескольких видов геометрических фигур:

квадратов;
прямоугольников;
параллелограммов;
окружностей.
Ваша задача – посчитать количество квадратов на каждом изображении из тестовых данных. Изображения для обучения можно скачать здесь.

## Описание данных
Весь набор данных состоит из 3 типов изображений в равном количестве:

Содержат только квадраты, которые могут иметь общие стороны, вкладываться друг в друга, но не пересекаться.
Содержат квадраты и другие фигуры, все они могут иметь общие стороны, вкладываться друг в друга, но не пересекаться.
Содержат квадраты и другие фигуры, которые дополнительно могут пересекаться.
Ниже находятся 2 файла, которые потребуются для решения задачи: train.csv – содержит информацию об изображениях для обучения, test.csv – содержит информацию об изображениях для теста. Имеют поля:

* img_path – путь до изображения
* label – число квадратов на изображении (только в обучающем наборе)
* type – тип изображения 1/2/3 (только в обучающем наборе)


Пример сабмита можно посмотреть в файле sample_submission.csv. Он содержит одну колонку label, порядок соответствует порядку данных в тестовом наборе.

## Метрика качества
В качестве метрики качества используется метрика Root Mean Square Error (RMSE):

## 0. Предварительная подготовка

### 0.0. Установим LightAutoML

In [1]:
# установим библиотеку LightAutoML с поддержкой энкодеров
# для текста и изображений на основе нейросетей
# !pip install -U lightautoml[all]

In [2]:
# чтобы работали нейросети и шаблон решения задач
# с картинками нужно поставить на библиотеку дополнительный фикс!
# !pip install LightAutoML-0.3.7-py3-none-any.whl[all]

In [3]:
# в качестве модели извлекающей признаки из картинок будет
# использоваться нейросеть EfficientNet
# установим эту зависимость.

# !pip install efficientnet-pytorch==0.7.0

# QUICK WORKAROUND FOR PROBLEM WITH PANDAS
# !pip install -U pandas

In [4]:
import os
import time

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import matplotlib.pyplot as plt

# LightAutoML шаблоны пресетов для задач комп зрения
from lightautoml.automl.presets.image_presets import TabularCVAutoML
from lightautoml.tasks import Task

/home/admin-gpu/anaconda3/envs/sber_lama/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
np.random.seed(42)
torch.set_num_threads(2)

### Загрузка данных

In [6]:
train = pd.read_csv('data/train.csv')

In [7]:
train.head()

,img_path,label,type
0,train_data/1.png,3,1
1,train_data/2.png,7,3
2,train_data/3.png,9,2
3,train_data/4.png,9,1
4,train_data/5.png,10,1


In [8]:
# добавим полный относительный путь до картинок из train
# в качестве новой колонки
path_to_dataset_train = 'data'
train['path'] =  path_to_dataset_train + '/' + train['img_path']

In [9]:
# колонка с имена картинок больше не нужна
train.drop(columns=['img_path'], inplace=True)

In [10]:
train.head()

,label,type,path
0,3,1,data/train_data/1.png
1,7,3,data/train_data/2.png
2,9,2,data/train_data/3.png
3,9,1,data/train_data/4.png
4,10,1,data/train_data/5.png


# Ставим задачу

In [15]:
# у нас задача регрессии, ставим метрику и лосс согласно условиям
# соревнования
task = Task('reg', loss='mse', metric='mse')

In [16]:
# целевая переменная для регрессии это PSNR
# переменная пути для картинок в столбце path
roles = {'target': 'label',
         'path' : ['path'],
         'drop' : ['type']
         }

In [17]:
automl = TabularCVAutoML(task=task,
                         timeout=5 * 3600,
                         cpu_limit=2,
                         reader_params={'cv': 5, 'random_state': 42})


# Обучение модели по кросс валидации по всей выборке

In [18]:
%%time 
# будет использовано 5 фолдов кросс валидации
# для извлечения признаков с картинок будет использована нейросеть EfficientNet b0
# после извлечения числовых признаков нейросетью на них будут учится стандартные модели
#  линейная регрессия и градиентный бустинг
oof_pred = automl.fit_predict(train, roles = roles, verbose = 4)

[08:04:39] Stdout logging level is DEBUG.
[08:04:39] Task: reg

[08:04:39] Start automl preset with listed constraints:
[08:04:39] - time: 18000.00 seconds
[08:04:39] - CPU: 2 cores
[08:04:39] - memory: 16 GB

[08:04:39] Train data shape: (3000, 3)

[08:04:39] Layer 1 train process start. Time left 18000.00 secs
Loaded pretrained weights for efficientnet-b0
[08:04:39] Load saved dataset for path
[08:04:39] Feature path transformed
[08:04:39] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[08:04:39] Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [], 'embed_sizes': (), 'data_size': 1280}
[08:04:39] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[08:04:39] Linear model: C = 1e-05 score = -2.3322192625777536
[08:04:39] Linear model: C = 5e-05 score = -1.9433305594713663
[08:04:39] Linear model: C 

In [19]:
# смотрим предсказания модели-регрессора
preds = oof_pred.data[:,0]
preds


array([4.322744  , 8.191117  , 8.26025   , ..., 5.8625326 , 6.945096  ,
       0.38278496], dtype=float32)

In [20]:
y_true = train['label'].values
y_true

array([3, 7, 9, ..., 7, 7, 1])

In [21]:
# Введем метрики оценки качества модели
def mse(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()
def mae(y_true, y_pred):
    return (np.abs(y_true - y_pred)).mean()

In [22]:
# Оценим нашу модель на валидационной выборке:
print(f"MSE = {mse(y_true, preds)}")
print(f"MAE = {mae(y_true, preds)}")

MSE = 1.5963662768334919
MAE = 0.9565228770921628


# Предскажем на тесте

In [23]:
test = pd.read_csv('data/test.csv')

In [24]:
test.head()

,img_path
0,test_data/1.png
1,test_data/2.png
2,test_data/3.png
3,test_data/4.png
4,test_data/5.png


In [26]:
# добавим полный относительный путь до картинок из test
# в качестве новой колонки
path_to_dataset_test = 'data'
test['path'] =  path_to_dataset_test + '/' + test['img_path']

In [27]:
# колонка с имена картинок больше не нужна
test.drop(columns=['img_path'], inplace=True)

In [28]:
test.head()

,path
0,data/test_data/1.png
1,data/test_data/2.png
2,data/test_data/3.png
3,data/test_data/4.png
4,data/test_data/5.png


In [29]:
# предсказание ансамблем моделей на тестовых данных
test_pred = automl.predict(test)

100%|██████████| 47/47 [00:04<00:00,  9.54it/s]

[08:07:43] Feature path transformed



/home/admin-gpu/anaconda3/envs/sber_lama/lib/python3.9/site-packages/lightautoml/image/image.py:84: RuntimeWarning: invalid value encountered in divide
  return list(hist / hist.sum())


In [30]:
test_pred = test_pred.data[:,0]

In [31]:
test_pred

array([ 7.4513717, 11.071525 ,  1.4318794, ..., 11.1494255,  6.2699213,
        3.389724 ], dtype=float32)

# публикуем сабмит

In [32]:
submission = pd.read_csv('data/sample_submission.csv')

In [33]:
submission

,label
0,0
1,0
2,0
3,0
4,0
...,...
5995,0
5996,0
5997,0
5998,0


In [34]:
submission['label'] = test_pred


In [35]:
submission

,label
0,7.451372
1,11.071525
2,1.431879
3,6.963997
4,7.484351
...,...
5995,10.608534
5996,9.632649
5997,11.149426
5998,6.269921


In [36]:
submission_path = 'submission/lightautoml.csv'

In [37]:
submission.to_csv(submission_path, index=False)